In [ ]:
import ee
import geemap

In [3]:
## import ipywidget
import ipywidgets as widgets
from ipyleaflet import WidgetControl
from ipywidgets import interact, interactive, fixed, interact_manual

In [ ]:
#import geemap and create an interactive map 
import geemap
Map = geemap.Map(center=[35.4387,139.7750], zoom=8)
Map = geemap.Map()
# Map
# Map.split_map(left_layer='HYBRID', right_layer='ROADMAP')
# Map
# Map.add_basemap('HYBRID')  # Add Google Map
# Map

In [7]:
#create ipywidgets 
# start date 
start_date = widgets.Text(
    value='',
    placeholder='YYYY-MM-DD',
    description='Date1:',
    disabled=False
)
# create ipywidgets 
end_date = widgets.Text(
    value='',
    placeholder='YYYY-MM-DD',
    description='Date2:',
    disabled=False
)

In [8]:
def on_button_clicked(b):
  # Confirmation of projection
  satellite = 'COPERNICUS/S2'
  #satellite = 'COPERNICUS/S2_SR'
  projection = ee.Projection('EPSG:32654') # set projection
  #specify the area 
  xmin=316550.8181000
  xmax=329683.2180000
  ymin=3881194.9193000
  ymax=3893174.1927000
  bl = ee.Geometry.Point([xmin, ymin],projection)
  #Specify the dataset and filtering parameters
  from_date= ee.Date(start_date.value)
  to_date  =ee.Date(end_date.value)
  dataset1= ee.ImageCollection(satellite).filterDate(from_date, from_date.advance(1, 'day')).filter(ee.Filter.geometry(bl)).filter(ee.Filter.equals('MGRS_TILE', '54SUD')).first()
  dataset2 = ee.ImageCollection(satellite).filterDate(to_date, to_date.advance(1, 'day')).filter(ee.Filter.geometry(bl)).filter(ee.Filter.equals('MGRS_TILE', '54SUD')).first()
  vis = {"bands": ["B3", "B2", "B1"], "min": 1000, "max": 3500}
  #now let us calculate the ndvi 
  nir1 = dataset1.select('B8')
  red1= dataset1.select('B4')
  ndvi_date1 = nir1.subtract(red1).divide(nir1.add(red1))
  nir2= dataset2.select('B8')
  red2= dataset2.select('B4')
  ndvi_date2 = nir2.subtract(red2).divide(nir2.add(red2))
  # ndvi difference 
  ndvi_diff = ndvi_date2.subtract(ndvi_date1)
  #now let us calculate the z score 
  # to calculate z -score of the ndvi difference
  ndvi_diff_stats = geemap.image_stats(ndvi_diff, scale=10)
  stats = ndvi_diff_stats.getInfo()
  # we need mean and standard deviation so first calculate the image statstics of ndvi 
  ndvi_mean = stats.get('mean').get('B8')
  ndvi_std = stats.get('std').get('B8')
  zscore = ndvi_diff.subtract(ndvi_mean).divide(ndvi_std)

  #define visualization parameters 
  vis_params_zscore = {
  'min': -1,
  'max': 1,
  'palette': ['blue','white','Red']}

  vis_params = {
  'min': -1,
  'max': 1,
  'palette': ['blue','white','green']}
  
  # add layers nature 
  # Map.addLayer(ndvi_date1, vis_params, 'NDVI_date1')
  # Map.addLayer(ndvi_date1, vis_params, 'NDVI_date2')


  Map.addLayer(dataset1, vis, 'Sentinel_Date1')
  Map.addLayer(dataset2, vis, 'Sentinel_Date2')
  Map.addLayer(ndvi_diff, vis_params, 'NDVI_difference')
  Map.addLayer(zscore, vis_params_zscore, 'Zscore')
  Map.centerObject(bl, 10)
  
  
  #below code is for sliders 
  # sentinel_layer = geemap.ee_tile_layer(dataset,vis,'Sentinel_image')
  # Map.split_map(left_layer='HYBRID', right_layer=sentinel_layer)

In [9]:
butt = widgets.Button(description='Apply')
out = widgets.Output()
butt.on_click(on_button_clicked) 

In [ ]:
output_widget = widgets.Output(layout={'border': '2px solid black'})
output_control = WidgetControl(widget=output_widget, position='bottomright')
Map.add_control(output_control)
with output_widget:
  output_widget.clear_output()
  display(start_date)
  display(end_date)
  display(butt)
Map